# Добро пожаловать в практическую часть


Вы находитесь в онлайн-сервисе Google Colab. Этот сервис используется для написания кода и предоставляет мощности для обучения моделей.

В рамках практической части мы пройдемся по основным этапам обучения модели. Весь код уже написан и логически разбит по ячейкам. Чтобы все отработало неоходимо последовательно запускать ячейки. Для запуска ячейки необходимо нажать круглую кнопку с треугольником в левом верхнем углу запускаемой ячейки.


---

### Нулевой шаг

Перед началом работы переключимся на использование мощностей видеокарты. Это позволит обучать модель значительно быстрее.

Для этого нужно нажать вверху "Изменить", затем "Настройки блокнота" и убедиться, что под словами "Аппартаный ускоритель" написано "GPU"

### Установка модулей

In [ ]:
# Первым делом нам нужно установить дополнительные модули для питона, которые помогут обучить нашу модель.
# pyTelegramBotAPI необходим для работы с телеграмботом
!pip install pyTelegramBotAPI
# transformers необходим для работы с моделями
!pip install transformers

# Для запуска ячейки необходимо нажать круглую кнопку с треугольником в левом верхнем углу запускаемой ячейки

## Активация необходимых модулей и доступа к гугл-диску

In [ ]:
# Какие-то модули уже стоят в питоне изначально, какие-то мы поставили выше. Сейчас их необходимо автивировать

import pandas as pd
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup,AutoTokenizer,AutoModelForCausalLM
from tqdm import tqdm, trange
import torch.nn.functional as F
import re

In [ ]:
# Разрешаем нашему коду смотреть в гугл-диск
from google.colab import drive
drive.mount('/content/drive')

## Исходные данные

In [ ]:
# Открываем текст
with open('/content/drive/MyDrive/Практическое_занятие/данные/anek.txt','r', encoding='utf-8') as f:
  opened_texts = f.read()
opened_texts = opened_texts.replace('<|startoftext|>','')
opened_texts[1400:1715]

In [ ]:
# Добавим к каждому тексту тег окончания текста, чтобы модель понимала где обычно они заканчиваются. 
texts = '<|endoftext|>'.join(['Анекдот: '+text for text in opened_texts.split('\n\n')][1:len(opened_texts.split('\n\n'))])

In [ ]:
# Проверим, как выглядит текст после изменений. Возьмем первые 1000 символов
texts[1400:1715]

In [ ]:
# Сохраним наши новые данные в файл
with open('/content/drive/MyDrive/Практическое_занятие/данные/a_text.txt','w', encoding='utf-8') as f:
  f.write(texts)

In [ ]:
# Посмотрим, что они хорошо сохранились
with open('/content/drive/MyDrive/Практическое_занятие/данные/a_text.txt','r', encoding='utf-8') as f:
  opened_texts = f.read()

opened_texts[1400:1715]

### Запускаем обучение модели

In [ ]:
%%time
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python /content/drive/MyDrive/Практическое_занятие/ru-gpts/pretrain_transformers.py \
    --output_dir=our_finetune_model \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=/content/drive/MyDrive/Практическое_занятие/данные/a_text.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 1 \
    --block_size 2048 \
    --overwrite_output_dir

## Генерируем текст

In [ ]:
model_name_or_path = "/content/our_finetune_model" # Путь до папки с нашей моделью
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path) # Активируем токенизатор - инуструмент, который превратит наш текст в нужный для модели формат
gpt = AutoModelForCausalLM.from_pretrained(model_name_or_path) # Активируем модель

### Запускаем генератор текста

Значение prompt отвечает за начало генерируемого текста. Его можно изменить, для лучшего результата сохранив тематику данных.

Параметры, которые можно позименять, чтобы изменить результат генерации:

*   max_length - максимальная длинна генерируемого текста в символах

*   num_return_sequences - сколько текстов сгенерирует модель. Чем больше текстов, тем дольше модель занимается генерацией

*   temperature - Чем выше этот параметр, тем более разнообразные тексты получаются. Однако, с повышением этого параметра повышается и вероятность, что текст получится менее похожим на человеческий

In [ ]:
prompt = 'Анекдот: Василий Иванович' # Начальные слова, которые подаются модели. С них и будет начинаться генерируемый текст

input_ids = tokenizer.encode(prompt, return_tensors="pt") # С помощью токенизатора преобразуем начальные слова в формат для модели

# Функция генерации текста
# Параметры, которые можно позименять, чтобы изменить результат генерации:
#   max_length - максимальная длинна генерируемого текста в символах
#   num_return_sequences - сколько текстов сгенерирует модель. Чем больше текстов, тем дольше модель занимается генерацией
#   temperature - Чем выше этот параметр, тем более разнообразные тексты получаются. 
#                 Однако, с повышением этого параметра повышается и вероятность, что текст получится менее похожим на человеческий
generated_text_samples = gpt.generate(
    input_ids,
    min_length=300,
    max_length=300,  
    num_return_sequences=5,
    no_repeat_ngram_size=3,
    repetition_penalty=1.5,
    top_p=1.,
    temperature=1,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

for i, beam in enumerate(generated_text_samples):
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=False).split('<|endoftext|>')[0].split('.')[0]))
  print()

Делаем ТГ Бота

Откройте мессенджер Telegram, войдите в вашу учетную запись.


1.   Введите в поле поиска @BotFather и выберите бота.

2.   Нажмите «Запустить» для активации бота BotFather. В ответ вы получите список команд по управлению ботов.

3.   Выберите или напечатайте и отправьте команду /newbot.

4.   Дайте имя боту — пользователи увидят это имя при общении с ботом. И никнейм бота — по нему можно будет найти бота в Telegram. Никнейм должен быть уникальным, не повторять существующие в базе и заканчиваться на слово «bot».

После того как вы выберите подходящее имя бот будет создан. Вы получите сообщение со ссылкой на бота t.me/<никнейм_бота>

Для подключения модели к боту нам понадобится токен


In [ ]:
import telebot
import torch
from transformers import AutoTokenizer,AutoModelForCausalLM
from transformers import GPT2LMHeadModel, GPT2Tokenizer


def model(text, mo, max_length=42):
  input_ids = tokenizer.encode(text, return_tensors="pt")

  generated_text_samples = mo.generate(
      input_ids, 
      min_length=100,
      max_length=150,  
      num_return_sequences=1,
      no_repeat_ngram_size=3,
      repetition_penalty=1.5,
      top_p=1.,
      temperature=1,
      do_sample=True,
      top_k=125,
      early_stopping=True
  )
  return tokenizer.decode(generated_text_samples[0], skip_special_tokens=False).split('<|endoftext|>')[0].split('.')[0]

bot = telebot.TeleBot('') # В этой строчке необходимо вставить токен от Вашего бота

@bot.message_handler(commands=["start"])
def start(m, res=False):
    bot.send_message(m.chat.id, 'Пожалуйста, напишите начало анекдота')

@bot.message_handler(content_types=["text"])
def handle_text(message):
    bot.send_message(message.chat.id, model(message.text, gpt))
    bot.send_message(message.chat.id, 'Напишите, пожалуйста, начало следующего анекдота')

bot.polling(none_stop=True, interval=0)